In [4]:
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import glob as glob
import numpy as np
import warnings
from scipy.stats import ttest_ind,f_oneway

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.utils.multiclass import unique_labels

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

warnings.filterwarnings(action='once')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

#divide into train and test set
TRAIN_START_DATE='2019-04-01 00:00:00'
TRAIN_END_DATE='2019-08-31 00:00:00'
TEST_START_DATE='2019-09-01 00:00:00'
TEST_END_DATE='2019-11-14 00:00:00'
ng_dist_shiftx_df=day_level_agg_df
XGB_Train = ng_dist_shiftx_df[(ng_dist_shiftx_df.index >= TRAIN_START_DATE) &
                                  (ng_dist_shiftx_df.index < TRAIN_END_DATE)]
XGB_Test = ng_dist_shiftx_df[(ng_dist_shiftx_df.index >= TEST_START_DATE) &
                                (ng_dist_shiftx_df.index < TEST_END_DATE)]

XGB_Train.to_csv('TwoStage_PredictiveModel/Predictive_Model_XGB_Data/Classification_XGB_Train_Apr_Sep2019.csv')
XGB_Test.to_csv('TwoStage_PredictiveModel/Predictive_Model_XGB_Data/Classification_XGB_Test_Apr_Sep2019.csv')
print("Training data--Test data Date split")
print(XGB_Train.shape, XGB_Test.shape)  
print("\n")
# Create X, y datasets and train,Val datasets
X = XGB_Train[XGB_Train.columns[2:-1]]
#X = XGB_Train['Right1_min']
y = XGB_Train['NG']

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.25,random_state=0)
# X_train = np.array(X_train).reshape(-1,1)
# X_val = np.array(X_val).reshape(-1,1)
# y_train = np.array(y_train).reshape(-1,1)
# y_val = np.array(y_val).reshape(-1,1)
#, X_val, y_train, y_val
features = X_train.columns
print("Training data--Validation data X-y split")
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print(features)
print("training data NG sample percentage",100*y_train.value_counts()[1]/sum(y_train.value_counts()))
print("testing data NG sample percentage",100*y_val.value_counts()[1]/sum(y_val.value_counts()))

#Fitting XGB Classifier 
model = xgb.XGBClassifier(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=1000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
model.fit(X_train,y_train)
y_pred = model.predict(data=X_val)
print (model)

actual = y_val
predicted = y_pred
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(actual, predicted))
print('Report : ')
print(classification_report(actual, predicted))


In [ ]:
from hyperopt import hp, fmin, tpe
from sklearn.metrics import mean_squared_error

X = all_df[all_df.columns[:-1]]
y = all_df[RESPONSE]
print(X.shape, y.shape)

space = {
    'n_estimators': hp.choice('n_estimators', range(1, 500)),
    'max_depth': hp.choice('max_depth', range(3, 100)),
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01),
    'silent': 1,
    'objective': 'multi:softprob',
    'nthread': -1, # set -1 to use all available scores
    'gamma': hp.quniform('gamma', 0.1, 2, 0.01),
    'min_child_weight': hp.quniform('min_child_weight', 0, 5, 0.5),
    'subsample': hp.quniform('subsample', 0.1, 0.9, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 0.9, 0.05),
    'colsample_bylevel': hp.quniform('colsample_bylevel', 0.1, 0.9, 0.05),
    #'colsample_bynode': hp.quniform('colsample_bynode', 0.1, 0.9, 0.05),
    'reg_alpha' : hp.quniform('reg_alpha', 0, 1, 0.00001), # this parameter is for L1 regulation
    'reg_lambda' : hp.quniform('reg_lambda', 0, 20, 0.1)
}

def objective(space):
    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.25,random_state=0)
    model = xgb.XGBClassifier(
            colsample_bylevel=space['colsample_bylevel'],
            colsample_bytree=space['colsample_bytree'],
            #colsample_bynode=space['colsample_bynode'],
            gamma=space['gamma'],
            learning_rate=space['learning_rate'],
            max_depth=space['max_depth'],
            min_child_weight=space['min_child_weight'],
            n_estimators=space['n_estimators'],
            nthread=space['nthread'],
            objective=space['objective'],
            random_state=0,
            reg_alpha=space['reg_alpha'], 
            reg_lambda=space['reg_lambda'],
            scale_pos_weight=1, 
            seed=None,
            silent=space['silent'],
            subsample=space['subsample'],
            verbosity=1
    )
    model.fit(X_train, y_train)
    return mean_squared_error(y_val,model.predict(X_val))



best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=100
)
print(best)

best_model = xgb.XGBClassifier(
            colsample_bylevel=best['colsample_bylevel'],
            colsample_bytree=best['colsample_bytree'],
            #colsample_bynode=best['colsample_bynode'],
            gamma=best['gamma'],
            learning_rate=best['learning_rate'],
            max_depth=best['max_depth'],
            min_child_weight=best['min_child_weight'],
            n_estimators=best['n_estimators'],
            random_state=0,
            reg_alpha=best['reg_alpha'], 
            reg_lambda=best['reg_lambda'],
            scale_pos_weight=1, 
            seed=None,
            subsample=best['subsample'],
            verbosity=1
)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_val)
actual = y_val
predicted = y_pred
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',f1_score(actual, predicted, average='micro'))
print('Report : ')
print(classification_report(actual, predicted))
